## ReMKiT1D input generator - Gaussian advection with Matrix-free discretization

This notebook revisits the ReMKiT1D_advection_test notebook, instead using the `"shift"` procedure of `UnaryTransform` alongside the `Node` and `treeDerivation` objects to define the Divergence and Gradient operators without the use of Matrix or Stencil terms. It demonstrates:
- Central difference discretization at the Python level.
- Non-uniform spatial (x) grid.
- (new in v1.2.0) Grid helper functions for dual cell widths and cell volumes.
- (new in v1.2.0) Support for the SUNDIALS `CVODE` integrator. Basic functionality is provided in this example.

This potentially allows for flexible top-level construction of matrices and boundary conditions without needing to be defined in the low-level Fortran library.

In [ ]:
import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid ,Node ,treeDerivation ,UnaryTransform
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.calculation_tree_support as ct

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Wrapper initialization


In [ ]:
rk = RKWrapper()

### Global parameters for writing the files


In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_advection_matrix_free/"
rk.setHDF5Path(hdf5Filepath) # The input and output location of any HDF5 files used/generated by the code

### Setting options for external libraries used by ReMKiT1D


#### MPI


In [ ]:
numProcsX = 4 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic direction 
haloWidth = 1 # Halo width in cells 
numProcs = numProcsH * numProcsX
rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization


In [ ]:
rk.setNormDensity(1.0e19) #n_0
rk.setNormTemperature(10.0) #T_0
rk.setNormRefZ(1.0) # reference ion charge for e-i collision time

### Grid initialization

In [ ]:
# Switch between uniform and non-uniform grids to see how the result compares.
xGridIsUniform = True

In [ ]:
xWidth = 0.025 # Uniform grid spacing.
xGridHalfSize = 256 # Total number of points in each half of the x-grid.

In [ ]:
if xGridIsUniform:
    # Uniformly-spaced x grid:
    xGridWidths = xWidth*np.ones(2*xGridHalfSize)
else:
    # Non-uniform x grid with wider spacing at the boundaries:
    xGridWidths = np.zeros(2*xGridHalfSize)
    xGridWidths[0:xGridHalfSize] = np.geomspace(4*xWidth,xWidth/10,xGridHalfSize)
    xGridWidths[xGridHalfSize:] = np.geomspace(xWidth/10,4*xWidth,xGridHalfSize)

# In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
vGrid = np.ones(1)
lMax = 0
gridObj = Grid(xGridWidths, vGrid, lMax, interpretXGridAsWidths=True)

rk.grid = gridObj

### Variables
Physical variables including density, temperature and flow are defined as before:


In [ ]:
n = 1 + np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) # A Gaussian perturbation
rk.addVarAndDual('n',n,isCommunicated=True)

T = np.ones(len(gridObj.xGrid)) # Constant temperature
rk.addVar('T',T,isDerived=True) # isDerived removes the variable from the implicit vector
rk.addVarAndDual('u',isCommunicated=True,primaryOnDualGrid=True) # primaryOnDualGrid denotes that the main variable is u_dual, and u is interpolated 

# Note: In v1.2.0, the "time" scalar is added automatically unless the wrapper is instructed otherwise.

In [ ]:
nNode = Node('n')
uNode = Node('u')
TNode = Node('T')

massRatio = 1/1836

wNode = 1.5*nNode*TNode + uNode**2/(nNode*massRatio) # assuming normalization to n_0*e*T_0

In [ ]:
rk.addCustomDerivation("wDeriv",treeDerivation(wNode)) # Registering the derivation in the wrapper

In [ ]:
rk.addVar("W",isDerived=True,derivationRule=sc.derivationRule("wDeriv",['n','u','T'])) # the derivation rule states which variables W depends on

Alongside the physical variables, we define stationary grid variables containing deep copies of data belonging to `rk.grid`:

In [ ]:
# Inverse dual grid cell widths, 1/dx[i+1/2] (center-to-center width; so exists on the dual grid):
# Extend the width at the outermost boundary cells.
invdx = 1/rk.grid.dualXGridWidths(extendedBoundaryCells=True)
rk.addVarAndDual("invdx",invdx,primaryOnDualGrid=True,isStationary=True,isCommunicated=True)

# Inverse grid cell volumes, 1/V[i]:
# Define two variables for the left and right-hand stencil points and apply left and right boundary conditions (1/V = 0) respectively.
invV = 1/rk.grid.xGridCellVolumes()
invV[0] = 0.
rk.addVar("invV_L",invV,isDerived=True,isCommunicated=True)

invV = 1/rk.grid.xGridCellVolumes()
invV[-1] = 0.
rk.addVar("invV_R",invV,isDerived=True,isCommunicated=True)

# Cell face Jacobians:
# This array is one point larger than the Grid.
# There is no need to use J at the first dual grid point (0th index), so it is discarded to allow J to fit on the dual grid:
J = rk.grid.xJacobian[1:]
rk.addVarAndDual("J",J,primaryOnDualGrid=True,isDerived=True,isCommunicated=True)

### Models

As before, the advection equation is given by:

$\frac{\partial n}{\partial t} = - \frac{\partial u}{\partial x}$

Instead of defining a Matrix and its stencil explicitly, we recreate the divergence stencil based on Eq.(15) of the ReMKiT1D paper:

$S_{ij} u_j = \frac{J_{i+\frac{1}{2}} \cdot u_{i+\frac{1}{2}} - J_{i-\frac{1}{2}} \cdot u_{i-\frac{1}{2}}}{V_i}$

for flow $u$, cell Jacobian $J_i$ and cell volume $V_i$, but this time using `Node` and `UnaryTransform("shift")` objects to fetch values in $J$ and $u$ from adjacent cells of a given grid.

Computationally, $u_{i-\frac{1}{2}}$ is equivalent to the $i$-th value in `Node('u_dual')`, while $u_{i+\frac{1}{2}}$ is found by shifting forward by one cell:

In [ ]:
newModel = sc.CustomModel("nAdvection")

# Define the central difference divergence stencil as a function:
def divNode(var_dual, customNormConst, J_dual, invV_L, invV_R):
    """Divergence operator.

    Args:
        var_dual: Variable on the dual grid on which the Divergence operator is to be applied.
        customNormConst: Scalar normalisation constant.
        J_dual: Cell face Jacobian values (rk.grid.xJacobian) on the dual grid.
        invV_L: Reciprocal of cell volume on the regular grid for the left stencil point.
        invV_R: Reciprocal of cell volume on the regular grid for the right stencil point.
    """
    shiftFwd = UnaryTransform("shift",intParams=[+1])
    return customNormConst*(J_dual*var_dual*invV_R -shiftFwd(J_dual*var_dual)*invV_L)

# Define a Derivation from this function. Remember to pass customNormConst = -1:
div = divNode(Node('u_dual'),-1.,Node('J_dual'),Node('invV_L'),Node('invV_R'))
rk.addCustomDerivation("divNode",derivOptions=treeDerivation(div))

# We use getLeafVars() from RMK_Support.calculation_tree_support to find the list of variables used by the "divNode" derivation:
newModel.addTerm("divFluxTerm",sc.DerivationTerm('n',sc.derivationRule("divNode",ct.getLeafVars(div))))

rk.addModel(newModel) # Note: No need to specify newModel.dict() in v1.2.0+

Likewise, the pressure gradient operator is given by:

$m_i \frac{\partial u}{\partial t} = - \frac{\partial (nkT)}{\partial x}$

The gradient stencil based on Eq.(16) of the ReMKiT1D paper (amended here):

$S_{ij} u_j = \frac{u_{i+\frac{1}{2}} - u_{i-\frac{1}{2}}}{dx_{i+\frac{1}{2}}}$

...will be defined below using `UnaryTransform` and `Node` objects.

Here, $dx_{i+\frac{1}{2}}$ corresponds with `invdx_dual` defined earlier. For $u_{i-\frac{1}{2}}$, we shift backward by one cell from $i+\frac{1}{2}$:

In [ ]:
newModel = sc.CustomModel("pGrad")

def gradNode(var, customNormConst, invdx_dual):
    """Grad operator.

    Args:
        var: Variable on the regular grid on which the Grad operator is to be applied.
        customNormConst: Scalar normalisation constant.
        invdx_dual: Reciprocal of dual cell widths (on dual grid).
    """
    shiftBwd = UnaryTransform("shift",intParams=[-1])
    return customNormConst*(shiftBwd(var) -var)*invdx_dual

# Define a Derivation from this function. Remember to pass the normalization constant, -m_e/2m_i, to ensure m_e*v**2/2 = k*T_0:
grad = gradNode(Node('n'),-massRatio/2,Node('invdx_dual'))
rk.addCustomDerivation("gradNode",derivOptions=treeDerivation(grad))

newModel.addTerm("pGradTerm",sc.DerivationTerm('u_dual',sc.derivationRule("gradNode",ct.getLeafVars(grad))))

rk.addModel(newModel)

### Integrator options

Available options include `CVODE` and `RK`. Ideally, there should be little difference in the result if sensible integrator options are used.

Note: The Backward Euler integrator requires defined matrix objects and cannot be used for this "matrix-free" implementation.

In [ ]:
integratorOption = "RK"

In [ ]:
if integratorOption=="CVODE":
    rk.addIntegrator(integratorOption,sc.CVODEIntegrator(absTol=1e-12))

elif integratorOption=="RK":
    rk.addIntegrator(integratorOption,sc.rkIntegrator(3))

else:
    print('ERROR - Valid values for integratorOption = "CVODE", "RK"')

# Note: In v1.2.0, no need to specify active numbers of active groups in rk.setIntegratorGlobalData:
rk.setIntegratorGlobalData(initialTimestep=0.1) # in normalized time units

integratorStep = sc.IntegrationStep(integratorOption)

# Add active model groups to an integrator step with the following:
for tag in rk.modelTags(integrableOnly=True):
    integratorStep.addModel(tag,rk.activeGroups(tag),rk.activeGroups(tag))

rk.addIntegrationStep("Step"+integratorOption,integratorStep.dict())

In [ ]:
rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

In [ ]:
rk.addTermDiagnosisForVars('n')

### Create config file

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


### Compare with analytic solution

In [ ]:
wave_speed= np.sqrt(massRatio/2)
n_analytic=np.zeros((numFiles+1,gridObj.numX()))
times = loadedData.coords['time'].data
L = sum(xGridWidths)
for i in range(numFiles+1):
        leftPositionMod = (gridObj.xGrid-wave_speed*times[i]) % L
        leftPosition = np.where(leftPositionMod > 0,leftPositionMod,leftPositionMod+L)
        rightPosition = (gridObj.xGrid+wave_speed*times[i]) % L
        n_analytic[i,:] =1 + 0.5*(np.exp(-(leftPosition-np.mean(gridObj.xGrid))**2) + np.exp(-(rightPosition-np.mean(gridObj.xGrid))**2)) 


In [ ]:
dataName = 'n'

curveDict = {t: hv.Scatter(loadedData[dataName][{"time":t}],label='simulation').opts(marker='P',color='r',s=20.)*hv.Curve((gridObj.xGrid,n_analytic[t,:]),label='analytic result').opts(title=f't = {loadedData["time"].values[t]:.2f} '+loadedData.coords["time"].attrs["units"],fontscale=2, fig_size=150,linewidth=2.0,color='k',alpha=0.5) for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()